In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install fingerflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.5 MB/s eta 0:00:00


In [3]:
from .minutiae_net import MinutiaeNet
from .classify_net import ClassifyNet
from .core_net import CoreNet
from . import utils


class Extractor:
    def __init__(self, coarse_net_path, fine_net_path, classify_net_path, core_net_path):
        self.__extraction_module = MinutiaeNet(coarse_net_path, fine_net_path)
        self.__classification_module = ClassifyNet(classify_net_path)
        self.__core_detection_module = CoreNet(core_net_path)

    def extract_minutiae(self, image_data):
        preprocessed_image = utils.preprocess_image_data(image_data)

        extracted_points = self.__extraction_module.extract_minutiae_points(
            preprocessed_image['image'],
            preprocessed_image['original_image'])

        classified_points = self.__classification_module.classify_minutiae_points(
            preprocessed_image['original_image'],
            extracted_points)

        core_coordinates = self.__core_detection_module.detect_fingerprint_core(image_data)

        return {'core': core_coordinates, 'minutiae': classified_points}


ImportError: ignored

In [ ]:
from datetime import datetime
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
# from /content/drive/MyDrive/fingerflow-main/src/fingerflow/extractor/extractor.py
# # from fingerflow import extractor
# from fingerflow import extractor
from src.fingerflow import extractor
# from .content.drive.MyDrive.fingerflow_main.src.fingerflow.extractor.extractor import *


COARSE_NET_PATH = "/content/drive/MyDrive/fingerflow-main/CoarseNet.h5"
FINE_NET_PATH = "/content/drive/MyDrive/fingerflow-main/FineNet.h5"
CLASSIFY_NET_PATH = "/content/drive/MyDrive/fingerflow-main/ClassifyNet_6_classes.h5"
CORE_NET_PATH = '/content/drive/MyDrive/fingerflow-main/CoreNet.weights'

IMG_PATH = '/content/drive/MyDrive/fingerflow-main/102_4.tif'
RADIUS = 5


def get_minutiae_color(point_class):
    switcher = {
        0: (0, 0, 255),
        1: (0, 255, 0),
        2: (255, 0, 0),
        3: (0, 255, 255),
        4: (255, 255, ),
        5: (255, 0, 255)
    }

    return switcher.get(point_class)


current_datetime = datetime.now().strftime("%d%m%Y%H%M%S")

extractor = extractor.Extractor(COARSE_NET_PATH, FINE_NET_PATH, CLASSIFY_NET_PATH, CORE_NET_PATH)

image = cv2.imread(IMG_PATH)

extracted_data = extractor.extract_minutiae(image)

extracted_minutiae = extracted_data['minutiae']
core = extracted_data['core']

start_point = (core['x1'].values[0], core['y1'].values[0])
end_point = (core['x2'].values[0], core['y2'].values[0])
color = (0, 0, 255)

cv2.rectangle(image, start_point, end_point, color, 2)


if len(extracted_minutiae) > 0:
    for index, minutia in extracted_minutiae.iterrows():
        print(minutia)
        x = minutia['x']
        y = minutia['y']
        o = minutia['angle']
        minutia_class = minutia['class']

        start_point = (int(x-RADIUS), int(y-RADIUS))
        end_point = (int(x+RADIUS), int(y+RADIUS))

        color = get_minutiae_color(int(minutia_class))

        cv2.rectangle(image, start_point, end_point, color, 2)

        line_start_point = (int(x), int(y))
        line_end_point = (int(x + 20 * np.cos(o)), int(y + 20 * np.sin(o)))

        cv2.line(image, line_start_point, line_end_point, color, 2)

cv2_imshow( image)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite(f'extractor_single_img_test_{current_datetime}.png', image)


In [ ]:
import pickle
import cv2
import numpy as np
import pandas as pd
# from fingerflow import extractor

# COARSE_NET_PATH = "/home/jakub/projects/bp/minutiae_classificator/models/CoarseNet.h5"
# FINE_NET_PATH = "/home/jakub/projects/bp/minutiae_classificator/models/FineNet.h5"
# CLASSIFY_NET_PATH = "/home/jakub/projects/bp/minutiae_classificator/models/ClassifyNet.h5"
# CORE_NET_PATH = '/home/jakub/projects/dp/kernel-detector/trainings/19-12-2021-final/yolo-kernel_best.weights'

IMG_PATH = '/home/user/Downloads/fingerflow-main/scripts/extractor/102_4.tif'
DATA_PATH = 'extractor_output.txt'

MINUTIAE_NUM = 20

# extractor = extractor.Extractor(COARSE_NET_PATH, FINE_NET_PATH, CLASSIFY_NET_PATH, CORE_NET_PATH)

image = cv2.imread(IMG_PATH)

extracted_minutiae = extractor.extract_minutiae(image)

with open(DATA_PATH, 'wb') as f:
    pickle.dump(extracted_minutiae, f)


def get_correct_core_point(core_data):
    correct_core = core_data[core_data.score == core_data.score.max()]

    x = correct_core[['x1', 'x2']].mean(axis=1)
    y = correct_core[['y1', 'y2']].mean(axis=1)

    return pd.DataFrame({'x': x.values, 'y': y.values})


def get_n_nearest_minutiae(extracted_data, core_point):
    minutiae_data = extracted_data['minutiae']

    if len(core_point) == 0 or len(minutiae_data) < MINUTIAE_NUM:
        return pd.DataFrame()

    minutiae_data['core_distance'] = np.linalg.norm(
        minutiae_data[['x', 'y']].values - core_point.values, axis=1)

    return minutiae_data.sort_values(by=['core_distance']).reset_index(drop=True)


def get_minutiae_color(point_class):
    switcher = {
        0: (255, 255, 255),
        1: (0, 255, 0),
        2: (255, 0, 0),
        3: (0, 255, 255),
        4: (255, 255, ),
        5: (255, 0, 255)
    }

    return switcher.get(point_class)


def show_core_point(core):
    start_point = (core['x1'].values[0], core['y1'].values[0])
    end_point = (core['x2'].values[0], core['y2'].values[0])
    color = (0, 0, 255)

    cv2.rectangle(image, start_point, end_point, color, 2)


def show_minutiae_points(minutiae, core_point):
    radius = 10
    core = (int(core_point['x']), int(core_point['y']))

    for index, point in minutiae.iterrows():
        center = (int(point['x']), int(point['y']))

        print(center, index)
        cv2.circle(image, center, radius, get_minutiae_color(int(point['class'])), 1)

        if index < MINUTIAE_NUM:
            cv2.line(image, core, center, (0, 0, 255), 1)


with open(DATA_PATH, 'rb') as f:
    extracted_minutiae = pickle.load(f)

    core_point = get_correct_core_point(extracted_minutiae['core'])
    enriched_minutiae = get_n_nearest_minutiae(extracted_minutiae, core_point)

    show_core_point(extracted_minutiae['core'])
    show_minutiae_points(enriched_minutiae, core_point)

    print(enriched_minutiae.to_markdown())
    cv2.imshow('fingerpint', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    cv2.imwrite('visualised_feature_vector_3.png', image)


In [ ]:
import os
import cv2
import numpy as np
# USING PYPI PACKAGE
# from fingerflow import extractor
# USING LOCAL VERSION - move script ro parent folder to run it
from fingerflow import extractor

# COARSE_NET_PATH = "/home/jakub/projects/bp/minutiae_classificator/models/CoarseNet.h5"
# FINE_NET_PATH = "/home/jakub/projects/bp/minutiae_classificator/models/FineNet.h5"
# CLASSIFY_NET_PATH = "/home/jakub/projects/bp/minutiae_classificator/models/ClassifyNet.h5"
# CORE_NET_PATH = '/home/jakub/projects/dp/kernel-detector/trainings/19-12-2021-final/yolo-kernel_best.weights'
# COARSE_NET_PATH = "/home/user/Downloads/fingerflow-main/scripts/extractor/CoarseNet.h5"
# FINE_NET_PATH = "/home/user/Downloads/fingerflow-main/scripts/extractor/FineNet.h5"
# CLASSIFY_NET_PATH = "/home/user/Downloads/fingerflow-main/scripts/extractor/ClassifyNet_6_classes.h5"
# CORE_NET_PATH = '/home/user/Downloads/fingerflow-main/scripts/extractor/CoreNet.weights'

extractor = extractor.Extractor(COARSE_NET_PATH, FINE_NET_PATH, CLASSIFY_NET_PATH, CORE_NET_PATH)


def read_image(image_path):
    original_image = np.array(cv2.imread(image_path, 0))
    image_size = np.array(original_image.shape, dtype=np.int32) // 8 * 8
    image = original_image[:image_size[0], :image_size[1]]

    output = dict()

    output['original_image'] = original_image
    output['image_size'] = image_size
    output['image'] = image

    return output


def get_extracted_minutiae_data(image_path):
    image = np.array(cv2.imread(image_path, 0))

    extracted_minutiae = extractor.extract_minutiae(image)

    return extracted_minutiae


def get_extracted_minutiae(image_folder):
    minutiae_files = []

    for _, _, files in os.walk(image_folder):
        for file_name in files:
            file_path = image_folder + file_name

            minutiae_data = get_extracted_minutiae_data(file_path)
            print(f"{file_name} - processed => ")
            print(minutiae_data)

    #         file_name_without_extension = os.path.splitext(os.path.basename(file_name))[0]
    #         minutiae = Minutiae(file_name_without_extension, minutiae_data)
    #         minutiae_files.append(minutiae)
    #         print(minutiae_files)

    # return minutiae_files


fil=get_extracted_minutiae(
    '/content/drive/MyDrive/fingerflow-main/tests/DB1_B')


In [ ]:
print(fil)